In [1]:
# 여기서는 기존 사전훈련된 bert 모델에 추가적으로 MLM 학습 시키는 과정을 설명한다.
# => 사전은 기존 사전에 단어가 추가된 사전을 이용한다.(추가된 사전은 별도로 만들어야 함)
#
# 참고 사이트 
# https://www.fatalerrors.org/a/further-pre-training-of-chinese-language-model-bert-roberta.html
# 소스는 : https://github.com/zhusleep/pytorch_chinese_lm_pretrain 에 run_language_model_bert.py 참조함
# MLM 다른 소스는 : https://towardsdatascience.com/masked-language-modelling-with-bert-7d49793e5d2c 참조 바람

import transformers
import numpy as np
import torch
from os import sys
sys.path.append('..')
from myutils import seed_everything, TextDatasetForNextSentencePrediction, GPU_info, mlogging
from transformers import BertTokenizer, BertModel, BertTokenizerFast, BertConfig, AutoModelWithLMHead, BertForMaskedLM

In [2]:
# 변수들 설정

# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
input_corpus = "../../data11/my_corpus/moco-corpus-kowiki2022.txt"

# 기존 사전훈련된 모델
input_model_path = "../../data11/model/bert/bert-multilingual-cased/"
# 기존 사전 + 추가된 사전 파일
vocab_file="../tokenizer_sample/moco-vocab/moco-corpus-kowiki202206-42000/vocab.txt"
# 출력 모델 저장 경로
ouput_model_dir = "../../data11/model/bert/bmc-moco-corpus-kowiki202206-42000"

# 토큰활 할때 최대 길이 
token_max_len = 130

# 훈련용 변수
batch_size = 32   # 64 하면, GPU Out of memory 발생함(=>**따라서 32 진행)
train_epochs = 10
learning_rate = 5e-5  # trainer로 훈련할때 기본이 5e-5임
 # embedding size 최대 몇 token까지 input으로 사용할 것인지 지정(기본:512) 512, 1024, 2048 식으로 지정함, 엄청난 장문을 다룰경우 10124까지
#max_position_embeddings = 128 
logging_steps = 10000  # 훈련시, 로깅할 step 수 (크면 10000번 정도하고, 작으면 100번정도)
save_steps = 50000     # 10000 step마다 모델 저장
save_total_limit = 2 # 마지막 3개 모델 빼고 과거 모델은 삭제(100000번째마다 모델 저장하는데, 마지감 3개 빼고 나머지는 삭제)

# NSP 관련 변수 (*여기서는 필요없음)
#NSP_block_size = 140

In [3]:
cuda = GPU_info()
print(cuda)

#seed 설정
seed_everything(111)

#logging 설정
logger =  mlogging(loggername="bertfpt", logfilename="bertfpt")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:bertfpt_2022-09-07.log


In [4]:
# tokeinzier 생성
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨
'''
tokenizer = BertTokenizer(vocab_file=vocab_file, 
                          max_len=token_max_len, 
                          do_lower_case=False)
'''


#tokenizer = BertTokenizerFast(vocab_speical_path)
tokenizer = BertTokenizerFast(
    vocab_file=vocab_file,
    max_len=token_max_len,
    do_lower_case=False,
    )


# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('special_token_size: {}, tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('vocab_size: {}'.format(vocab_size))
print('tokenizer_len: {}'.format(len(tokenizer)))

special_token_size: 5, tokenizer.vocab_size: 152537
vocab_size: 152538
tokenizer_len: 152537


In [5]:
# 모델 생성
#output_hidden_states = False # 기본은 False=>output 2개 출력됨, True로 지정하면 output이 3개 출력됨
#return_dict = True   #False로 지정하는 경우 일반적인 tuple을 리턴, True인 경우는 transformers.file_utils.ModelOutput(ouput.logisc) 으로 리턴
#model = BertModel.from_pretrained(input_model_path, output_hidden_states = output_hidden_states, return_dict = return_dict)

# AutoModelWithLMHead 대신에 -> AutoModelForMaskedLM, BertForMaskedLM 해도됨
#=>원래 AutoModelWithLMHead 하면 내부적으로 BertForMaskedLM 가 로딩됨

if input_model_path:
    # further pre-training 인 경우 (기존 거에 추가적으로 하는 경우)
    config = BertConfig.from_pretrained(input_model_path)
    
    model = AutoModelWithLMHead.from_pretrained(input_model_path,
                                                from_tf=bool(".ckpt" in input_model_path),
                                                config=config) 
    print('further pre-training')
else:
    # Training new model from scratch 인 경우 (완전 새롭게 모델을 만드는 경우)
    model = AutoModelWithLMHead.from_config(config)
    print('Training new model from scratch')
 
#################################################################################
# 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
# 재설정하지 않으면, 다음과 같은 에러 발생함
# CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
#  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
#
#     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
#     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
#################################################################################
model.resize_token_embeddings(len(tokenizer))

# 훈련모드로 변경(평가모드 : model.eval())
model.train()



/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:998: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at ../../data11/model/bert/bert-multilingual-cased/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

further pre-training


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(152537, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          

In [6]:
model.num_parameters()

203343833

In [7]:
# MLM(Markup Language Model), NSP(Next Sentence Prediction) 구성
from transformers import DataCollatorForLanguageModeling, TextDataset
from myutils import MyTextDataset, MyLineByLineTextDataset

'''
# NSP 만들기
train_dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path=input_corpus,
    block_size=NSP_block_size,
    overwrite_cache=False,
    short_seq_probability=0.1,
    nsp_probability=0.5,
)
'''

# further-pretrain 일때는 일단 NSP는 입력안함.
# => 따라서 입력 corpus에 대해, NSP Dataset 이 아니라, TextDataset 으로 만듬
# => 라인 구분된 말뭉치는 MyLineByLineTextDataset 사용, 라인 구분없는 말뭉치는 MyTextDataset 이용, 

#train_dataset = MyTextDataset(tokenizer=tokenizer, file_path=input_corpus, block_size=token_max_len, overwrite_cache=True)
train_dataset = MyLineByLineTextDataset(tokenizer=tokenizer, file_path=input_corpus, block_size=token_max_len)
print(train_dataset)

Creating features from dataset file at ../../data11/my_corpus/moco-corpus-kowiki2022.txt
==>[Start] file read lines: ../../data11/my_corpus/moco-corpus-kowiki2022.txt


  0%|          | 0/7680008 [00:00<?, ?it/s]

<==[End] file read lines: ../../data11/my_corpus/moco-corpus-kowiki2022.txt
==>[Start] tokenizer
<==[End] tokenizer


In [8]:
# NSP 출력 해보기
# => MyLineByLineTextDataset 인 경우에는 example['input_ids'] 로 MyTextDataset 인 경우엔는 example로 해줘야 함.
count = 0
for example in train_dataset:
    token_str = [[tokenizer.convert_ids_to_tokens(s) for s in example['input_ids'].tolist()]]
    print(token_str)
    print('count:{}=>{}'.format(count,example))
    count +=1
    if count > 5:
        break

[['[CLS]', 'Refer', 'to', 'the', 'V', '$', 'SYSTEM', '_', 'EVENT', 'view', 'for', 'time', 'wait', '##ed', 'and', 'average', 'waits', 'for', 'thefollowing', 'actions', ':', '[SEP]']]
count:0=>{'input_ids': tensor([   101, 142678,  10114,  10105,    159,    109, 122321,    168, 149821,
         17904,  10142,  10635,  83279,  10336,  10111,  13551, 126341,  10142,
        124429,  22115,    131,    102])}
[['[CLS]', 'To', 'estimate', 'the', 'time', 'wait', '##ed', 'for', 'reads', 'in', '##cu', '##rre', '##d', 'by', 're', '##read', '##ing', 'data', 'blocks', 'that', 'had', 'tobe', 'written', 'to', 'disk', 'because', 'of', 'a', 'request', 'from', 'another', 'instance', ',', 'multi', '##ply', 'the', 'stati', '##stic', '(', 'for', 'example', ',', 'the', 'time', 'wait', '##ed', 'for', 'db', 'ﬁ', '##le', 'sequential', 'reads', ')', 'by', 'the', 'percentage', 'of', 'read', '##I', '/', 'O', 'caused', 'by', 'previous', 'cache', 'ﬂ', '##ush', '##es', 'as', 'shown', 'in', 'this', 'formula', ':', '[

In [9]:
# MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
'''
# MLM 출력 해보기
for example in train_dataset:
    print(type(example))
    mlm_sample = data_collator(example['input_ids'])
    token_str = [[tokenizer.convert_ids_to_tokens(s) for s in mlm_sample['input_ids']]]
    print(token_str)
    print('count:{}=>{}'.format(count,mlm_sample))
    count +=1
    if count > 5:
        break
'''        
# MLM 출력 해보기
mlm_sample = data_collator(train_dataset.examples[2:3])
print(type(mlm_sample))
print(mlm_sample.keys())
token_str = [[tokenizer.convert_ids_to_tokens(s) for s in mlm_sample['input_ids']]]
print(token_str)
print(mlm_sample)


<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'attention_mask', 'labels'])
[[['[CLS]', 'Where', '"', 'lock', 'buffers', 'for', 'read', '"', 'is', '[MASK]', '[MASK]', 'for', 'lock', 'converts', 'from', 'N', 'to', 'S', 'derived', 'from', '##V', '$', 'L', '##OCK', '_', 'ACT', '##IV', '##IT', '##Y', '[MASK]', '"', '[MASK]', '[MASK]', '"', 'is', 'from', 'the', 'V', '$', 'SYS', '##ST', '##AT', 'view', '.', '[SEP]']]]
{'input_ids': tensor([[   101,  23525,    107,  79601, 125495,  10142,  24944,    107,  10124,
            103,    103,  10142,  79601, 127838,  10188,    151,  10114,    156,
          29369,  10188,  11779,    109,    149,  96608,    168,  90119,  91238,
          37611,  14703,    103,    107,    103,    103,    107,  10124,  10188,
          10105,    159,    109, 125166,  32995,  32071,  17904,    119,    102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1

In [10]:
from transformers import Trainer, TrainingArguments
import os
# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir=ouput_model_dir,
    overwrite_output_dir=True,
    num_train_epochs=train_epochs,
    per_gpu_train_batch_size=batch_size,
    learning_rate=learning_rate,                      # lr: 기본 5e-5
    save_steps=save_steps,    # step 수마다 모델을 저장
    save_total_limit=save_total_limit, # 마지막 두 모델 빼고 과거 모델은 삭제
    logging_steps=logging_steps
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    train_dataset=train_dataset   #TEXT 혹은 NSP(Next Setence Predictions)
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [11]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7679990
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2400000
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# 학습한 모델 저장
trainer.save_model(ouput_model_dir)

In [ ]:
# tokeinizer 파일 저장(vocab)
tokenizer.save_pretrained(ouput_model_dir)